In [ ]:
from glob import glob
import pyarrow.parquet as pq
import pandas as pd
ROOT_DIR = "/.cache/modelscope/datasets/BAAI/Infinity-Instruct/7M"
all_files = glob(ROOT_DIR+"/*")
print(len(all_files), all_files)
from tqdm import tqdm
import json
from transformers import AutoTokenizer
model_path = "/.cache/modelscope/models/Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [ ]:

def to_alpaca_data(str_json_data):
    return {
                "instruction": str_json_data[-2]["value"],
                "input": "",
                "output": str_json_data[-1]["value"]
            }

def to_share_gpt_data(str_json_data):
    conversations = str_json_data.tolist()
    text = ""
    for item in conversations:
        text += item["value"]
    return {
        "conversations": conversations
    }, text

def process_one_data(file_dir):
    table = pq.read_table(file_dir)
    df = table.to_pandas()
    # Index(['id', 'conversations', 'label', 'langdetect', 'source'], dtype='object')
    # print(df.columns)
    # 可以过滤=======================
    # need_idx = df["langdetect"] == "zh-cn"
    # df = df.loc[need_idx, :]
    #===============================
    str_json_datas = df["conversations"]
    language_datas = df["langdetect"]
    muti_turn_counter = 0
    odd_cov_counter = 0
    encounter = 0
    a_file_datas = []
    text_list = []
    for idx, str_json_data in  enumerate(str_json_datas):
        L = len(str_json_data)
        # #! 过滤出中文数据集
        # if language_datas.iloc[idx] != "zh-cn":
        #     encounter += 1
        #     continue
        #! 过滤多轮对话
        if L > 2:
            muti_turn_counter += 1
        if L % 2 == 1:
            odd_cov_counter += 1
        # a_file_datas.append(to_alpaca_data(str_json_data)) 
        conversation, text= to_share_gpt_data(str_json_data)
        text_list.append(text)
        a_file_datas.append(conversation)
    
    input_ids = tokenizer(text_list)['input_ids']
    token_num_list = [len(x) for x in input_ids]
    print(f"all data:{len(str_json_datas)} muti_turn_counter:{muti_turn_counter} odd_cov_counter:{odd_cov_counter} encounter:{encounter}")
    return a_file_datas,token_num_list

all_datas = []
token_nums = []
for file_dir in tqdm(all_files):
    datas,token_num_list = process_one_data(file_dir) 
    all_datas.extend(datas) 
    token_nums.extend(token_num_list)
print(len(all_datas))
print(len(token_nums))

In [ ]:
import numpy as np
np.percentile(token_nums,[50, 60, 70, 80, 90,95,99])
print(np.percentile(token_nums,[50, 60, 70, 80, 90,95, 99]))
filtered_long_data = []
filtered_token_num = []
for i in range(len(all_datas)):
    if token_nums[i] < 768:
        filtered_long_data.append(all_datas[i])
        filtered_token_num.append(token_nums[i])
print(np.percentile(filtered_token_num,[50, 60, 70, 80, 90,95, 99]))
print(len(filtered_token_num))

In [ ]:
import os
file_path = '/.cache/sft_data/llamafactory_input/BAAI_Infinity-Instruct_682W.jsonl'

directory = os.path.dirname(file_path)

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"目录{directory} 不存在，已创建")
else:
    print(f"目录{directory} 已存在")

with open(file_path, 'w', encoding="utf-8") as f:
    for item in filtered_long_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')